In [44]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import os
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.decomposition import PCA

In [45]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM, GRU
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.metrics import RootMeanSquaredError,mean_absolute_percentage_error
from keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout
from keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [46]:
# split a univariate time series into patterns
def get_Patterns(TSeries, n_inputs,h):
    X,y,z = pd.DataFrame(np.zeros((len(TSeries)-n_inputs-h+1,n_inputs))), pd.DataFrame(), pd.DataFrame()
    for i in range(len(TSeries)):
        # find the end of this pattern
        end_ix = i + n_inputs + h - 1
        # check if we are beyond the time series
        if end_ix > len(TSeries)-1:
            break
        # gather input and output parts of the pattern
        for j in range(n_inputs):
            X.loc[i,j]=TSeries.iloc[i+j,0]
        i=i+n_inputs
        y=y.append(TSeries.iloc[end_ix], ignore_index = True)
    return np.array(X),np.array(y)

In [47]:
# originalData should be a Column Vectored DataFrame
def minmaxNorm(originalData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    normalizedData=np.zeros(lenOriginal)   
    normalizedData = []
    for i in range (lenOriginal):
        normalizedData.append((originalData.iloc[i]-min2norm)/(max2norm-min2norm))    
    return pd.DataFrame(normalizedData)

In [48]:
# originalData and forecastedData should be Column Vectored DataFrames
def minmaxDeNorm( originalData, forecastedData, lenTrainValidation):
    max2norm=max(originalData.iloc[0:lenTrainValidation,0])
    min2norm=min(originalData.iloc[0:lenTrainValidation,0])
    lenOriginal=len(originalData)
    denormalizedData=[]    
    for i in range (lenOriginal):
        denormalizedData.append((forecastedData.iloc[i]*(max2norm-min2norm))+min2norm)  
    return pd.DataFrame(denormalizedData)

In [49]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findRMSE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainRMSE=0;
    for i in range (lenTrainValidation):
        trainRMSE=trainRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2) 
    trainRMSE=np.sqrt(trainRMSE/lenTrainValidation)

    testRMSE=0;
    for i in range (lenTrainValidation,l,1):
        testRMSE=testRMSE+np.power((forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]),2)
    testRMSE=np.sqrt(testRMSE/lenTest)
    return trainRMSE, testRMSE 

In [50]:
# Timeseries_Data and forecasted_value should be Column Vectored DataFrames
def findMAE( Timeseries_Data, forecasted_value,lenTrainValidation):
    l=Timeseries_Data.shape[0]
    lenTest=l-lenTrainValidation
    trainMAE=0;
    for i in range (lenTrainValidation):
        trainMAE=trainMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0]) 
    trainMAE=(trainMAE/(lenTrainValidation));

    testMAE=0;
    for i in range (lenTrainValidation,l,1):
        testMAE=testMAE+np.abs(forecasted_value.iloc[i,0]-Timeseries_Data.iloc[i,0])
    testMAE=(testMAE/lenTest);
    return trainMAE, testMAE

In [51]:
def Find_Features_LSTM(x,y,lenValid,lenTest):
    NOP=len(y)
    n_features=1
    lenTrain=NOP-lenValid-lenTest
    LagLength=x.shape[1]
    xTrain=x[0:lenTrain,:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:lenTrain,0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model1 = Sequential()
    model1.add(LSTM(256, activation='tanh', input_shape=(LagLength,n_features),kernel_initializer="he_normal",name="layer1"))
    model1.add(Flatten())
    model1.add(Dense(16, activation='relu',kernel_initializer="he_normal"))
    model1.add(Dense(1,activation='linear'))
    checkpoint_filepath = './'
    cp1=ModelCheckpoint(checkpoint_filepath,save_best_only='True')
    model1.compile(loss=MeanSquaredError(),optimizer=Adam(learning_rate=0.01),metrics=[RootMeanSquaredError()])
    # fit
    model1.fit(xTrain, yTrain,validation_data=(xValid,yValid), epochs=100, verbose=1,batch_size=16)
    yhatNorm=model1.predict(x).flatten().reshape(x.shape[0],1)
    feature_extractor = Model(inputs=model1.inputs, outputs=model1.get_layer(name="layer1").output)
    features = pd.DataFrame(feature_extractor(x))
    return features

In [52]:
def Find_Fitness(x,y,lenValid,lenTest,model):
    NOP=y.shape[0]
    lenTrain=NOP-lenValid-lenTest
    xTrain=x[0:(lenTrain+lenValid),:]
    xValid=x[lenTrain:(lenTrain+lenValid),:]
    xTest=x[(lenTrain+lenValid):NOP,:]
    yTrain=y[0:(lenTrain+lenValid),0]
    yValid=y[lenTrain:(lenTrain+lenValid),0]
    yTest=y[(lenTrain+lenValid):NOP,0]
    model.fit(xTrain, yTrain)
    yhatNorm=model.predict(x).flatten().reshape(x.shape[0],1)
    return pd.DataFrame(yhatNorm)

In [54]:
#Read the Time Series Dataset
Timeseries_Data=pd.read_csv('AQI.csv',header=None)
LagLength=24
h=1
NumberOfRepeatitions=2
lt=Timeseries_Data.shape[0]
lenTrain=int(round(lt*0.6))
lenValidation=int(round(lt*0.2))
lenTest=int(lt-lenTrain-lenValidation)
# NORMALIZE THE DATA
normalizedData=minmaxNorm(Timeseries_Data,lenTrain+lenValidation);
# Transform the Time Series into Patterns Using Sliding Window
X, y = get_Patterns(normalizedData, LagLength, h)
LSTMFeatures=Find_Features_LSTM(X,y,lenValidation,lenTest)
#X=np.hstack((LSTMFeatures,GRUFeatures))
#X = SelectKBest(score_func=f_regression, k=24).fit_transform(X, y)
X=np.array(LSTMFeatures)
pca = PCA(n_components=24)
X = pca.fit_transform(X)
MeanAccuracyT=pd.DataFrame()
StdAccuracyT=pd.DataFrame()
model=RandomForestRegressor()
name='9.RandomForestRegressor'
file1='./'+str(name)+"_Accuracy.xlsx"
file2='./'+str(name)+"_Forecasts.xlsx"
Forecasts=pd.DataFrame()
Accuracy=pd.DataFrame()
ynorm1=Find_Fitness(X,y,lenValidation,lenTest,model)
ynorm=pd.DataFrame(normalizedData.iloc[0:LagLength,0])
ynorm=ynorm.append(ynorm1,ignore_index = True)
yhat=minmaxDeNorm(Timeseries_Data, ynorm, lenTrain+lenValidation)
Accuracy.loc[0,0],Accuracy.loc[0,1]=findRMSE( Timeseries_Data,yhat,lenTrain+lenValidation)
Accuracy.loc[0,2],Accuracy.loc[0,3]=findMAE( Timeseries_Data,yhat,lenTrain+lenValidation)
Forecasts=Forecasts.append(yhat.T,ignore_index = True)
Accuracy.to_excel(file1,sheet_name='Accuracy',index=False)
Forecasts.T.to_excel(file2,sheet_name='Forecasts',index=False)
Accuracy

Epoch 1/100
18/18 [==============================] - 8s 181ms/step - loss: 2.4794 - root_mean_squared_error: 1.5746 - val_loss: 0.0077 - val_root_mean_squared_error: 0.0877
Epoch 2/100
18/18 [==============================] - 2s 102ms/step - loss: 0.0048 - root_mean_squared_error: 0.0690 - val_loss: 9.6834e-04 - val_root_mean_squared_error: 0.0311
Epoch 3/100
18/18 [==============================] - 2s 104ms/step - loss: 0.0011 - root_mean_squared_error: 0.0332 - val_loss: 3.5581e-04 - val_root_mean_squared_error: 0.0189
Epoch 4/100
18/18 [==============================] - 2s 93ms/step - loss: 0.0012 - root_mean_squared_error: 0.0353 - val_loss: 2.2692e-04 - val_root_mean_squared_error: 0.0151
Epoch 5/100
18/18 [==============================] - 2s 92ms/step - loss: 0.0013 - root_mean_squared_error: 0.0364 - val_loss: 3.1969e-04 - val_root_mean_squared_error: 0.0179
Epoch 6/100
18/18 [==============================] - 2s 95ms/step - loss: 6.6731e-04 - root_mean_squared_error: 0.0258 - 

Epoch 47/100
18/18 [==============================] - 2s 93ms/step - loss: 4.4562e-04 - root_mean_squared_error: 0.0211 - val_loss: 4.9837e-04 - val_root_mean_squared_error: 0.0223
Epoch 48/100
18/18 [==============================] - 2s 92ms/step - loss: 5.1031e-04 - root_mean_squared_error: 0.0226 - val_loss: 4.1943e-04 - val_root_mean_squared_error: 0.0205
Epoch 49/100
18/18 [==============================] - 2s 92ms/step - loss: 3.2738e-04 - root_mean_squared_error: 0.0181 - val_loss: 2.5206e-04 - val_root_mean_squared_error: 0.0159
Epoch 50/100
18/18 [==============================] - 2s 92ms/step - loss: 3.1738e-04 - root_mean_squared_error: 0.0178 - val_loss: 3.2389e-04 - val_root_mean_squared_error: 0.0180
Epoch 51/100
18/18 [==============================] - 2s 92ms/step - loss: 7.4666e-04 - root_mean_squared_error: 0.0273 - val_loss: 3.8484e-04 - val_root_mean_squared_error: 0.0196
Epoch 52/100
18/18 [==============================] - 2s 94ms/step - loss: 3.3336e-04 - root_me

18/18 [==============================] - 2s 91ms/step - loss: 8.7562e-04 - root_mean_squared_error: 0.0296 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0386
Epoch 93/100
18/18 [==============================] - 2s 89ms/step - loss: 8.2528e-04 - root_mean_squared_error: 0.0287 - val_loss: 0.0041 - val_root_mean_squared_error: 0.0639
Epoch 94/100
18/18 [==============================] - 2s 90ms/step - loss: 0.0011 - root_mean_squared_error: 0.0331 - val_loss: 2.9924e-04 - val_root_mean_squared_error: 0.0173
Epoch 95/100
18/18 [==============================] - 2s 89ms/step - loss: 6.8545e-04 - root_mean_squared_error: 0.0262 - val_loss: 9.3664e-04 - val_root_mean_squared_error: 0.0306
Epoch 96/100
18/18 [==============================] - 2s 97ms/step - loss: 5.2120e-04 - root_mean_squared_error: 0.0228 - val_loss: 4.5724e-04 - val_root_mean_squared_error: 0.0214
Epoch 97/100
18/18 [==============================] - 2s 94ms/step - loss: 5.1371e-04 - root_mean_squared_error: 0.0227 

,0,1,2,3
0,4.718429,6.380979,2.05765,3.071
